In [114]:
%matplotlib inline
import numpy as np 
import pandas as pd
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.cluster import MiniBatchKMeans
import warnings
from sklearn.ensemble import GradientBoostingRegressor
import seaborn as sns

In [115]:
train = pd.read_csv("train.csv")

In [148]:
test = pd.read_csv("data_for_test.csv")

In [117]:
submit = pd.read_csv("sample_submission.csv")

In [119]:
def preprocessing(df_train, df_test):
    """
    normalizing continuous
    adding features
    normalizing continuous variables
    """
    df_train[['pid','reading_count']] = df_train['key'].str.split('-', expand=True)
    df_train = df_train.drop('pid', axis=1)
    df_train['reading_count'] = df_train['reading_count'].astype(int)
    
    df_test[['pid','reading_count']] = df_test['key'].str.split('-', expand=True)
    df_test = df_test.drop('pid', axis=1)
    df_test['reading_count'] = df_test['reading_count'].astype(int)
       
    #normalize
    cols = ['xx1','xx2','xx3','xx4','xx5'] 
    for col in cols:
        df_test[col] = (df_test[col] - df_train[col].mean())/df_train[col].std()
        df_train[col] = (df_train[col] - df_train[col].mean())/df_train[col].std()
    return df_train, df_test

In [153]:
train, test = preprocessing(train, test)

In [122]:
train_MAP = train.drop(['y_mean_HR','key'], axis = 1)
train_HR = train.drop(['y_mean_MAP','key'], axis = 1)

In [123]:
valid_idx = range(int(train.shape[0]*80/100), train.shape[0])
train_idx = range(int(train.shape[0]*80/100))

In [124]:
valid_MAP = train_MAP.loc[valid_idx,:]
train_MAP = train_MAP.loc[train_idx,:]
valid_HR = train_HR.loc[valid_idx,:]
train_HR = train_HR.loc[train_idx,:]

## xgb on MAP

In [131]:
model = GradientBoostingRegressor(learning_rate=0.1, max_depth=7, n_estimators=50)
model.fit(train_MAP.drop('y_mean_MAP', axis=1), train_MAP['y_mean_MAP'])
model.score(valid_MAP.drop('y_mean_MAP', axis=1), valid_MAP['y_mean_MAP'])

0.7242357640461901

In [76]:
model1 = GradientBoostingRegressor(learning_rate=0.3, max_depth=7, n_estimators=50)
model1.fit(train_MAP.drop('y_mean_MAP', axis=1), train_MAP['y_mean_MAP'])
model1.score(valid_MAP.drop('y_mean_MAP', axis=1), valid_MAP['y_mean_MAP'])

0.6977873200279159

In [77]:
model2 = GradientBoostingRegressor(learning_rate=0.1, max_depth=10, n_estimators=50)
model2.fit(train_MAP.drop('y_mean_MAP', axis=1), train_MAP['y_mean_MAP'])
model2.score(valid_MAP.drop('y_mean_MAP', axis=1), valid_MAP['y_mean_MAP'])

0.706162649819891

## rf on MAP

In [78]:
from sklearn.ensemble import RandomForestRegressor

In [81]:
model_rf = RandomForestRegressor(n_estimators=100, min_samples_leaf=10,
                                 max_features='sqrt', random_state=42, verbose=True)
model_rf.fit(train_MAP.drop('y_mean_MAP', axis=1), train_MAP['y_mean_MAP'])
model_rf.score(valid_MAP.drop('y_mean_MAP', axis=1), valid_MAP['y_mean_MAP'])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.0s finished


0.7025599503610953

In [84]:
model_rf_1 = RandomForestRegressor(n_estimators=500, min_samples_leaf=5,
                                 max_features='sqrt', random_state=42, verbose=True, n_jobs=-1)
model_rf_1.fit(train_MAP.drop('y_mean_MAP', axis=1), train_MAP['y_mean_MAP'])
model_rf_1.score(valid_MAP.drop('y_mean_MAP', axis=1), valid_MAP['y_mean_MAP'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.5min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.6s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    1.5s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    1.8s finished


0.6958036260422567

### BEST model for MAP is xgb model

In [132]:
MAP_predictions = model.predict(test.drop('key', axis=1))

## xgb on HR

In [91]:
model_HR = GradientBoostingRegressor(learning_rate=0.1, max_depth=7, n_estimators=50)
model_HR.fit(train_HR.drop('y_mean_HR', axis=1), train_HR['y_mean_HR'])
model_HR.score(valid_HR.drop('y_mean_HR', axis=1), valid_HR['y_mean_HR'])

0.8922465360829545

In [92]:
model1_HR = GradientBoostingRegressor(learning_rate=0.1, max_depth=5, n_estimators=100)
model1_HR.fit(train_HR.drop('y_mean_HR', axis=1), train_HR['y_mean_HR'])
model1_HR.score(valid_HR.drop('y_mean_HR', axis=1), valid_HR['y_mean_HR'])

0.8821506145872234

In [93]:
model2_HR = GradientBoostingRegressor(learning_rate=0.1, max_depth=7, n_estimators=100)
model2_HR.fit(train_HR.drop('y_mean_HR', axis=1), train_HR['y_mean_HR'])
model2_HR.score(valid_HR.drop('y_mean_HR', axis=1), valid_HR['y_mean_HR'])

0.8706712503905317

## rf on HR

In [95]:
model_HR_rf = RandomForestRegressor(n_estimators=500, min_samples_leaf=5,
                                    max_features='sqrt', random_state=42, verbose=True, n_jobs=-1)
model_rf_1.fit(train_HR.drop('y_mean_HR', axis=1), train_HR['y_mean_HR'])
model_rf_1.score(valid_HR.drop('y_mean_HR', axis=1), valid_HR['y_mean_HR'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   29.7s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.5min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.6s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    1.3s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    1.6s finished


0.8441189773874381

### Best model for HR is xbg model

In [96]:
HR_predictions = model_HR.predict(test)

In [154]:
test['y_mean_MAP'] = MAP_predictions
test['y_mean_HR'] = HR_predictions

In [155]:
toSubmit = test[['key','y_mean_MAP','y_mean_HR']]

In [156]:
toSubmit = toSubmit.groupby('key').mean()

In [157]:
toSubmit.to_csv("no_one_1.csv")